# CoV Divergent Read Simulations
```
Lead     : ababaian
Issue    : na
start    : 2020 04 11
complete : YYYY MM DD
files    : ~/serratus/notebook/200411/
```

## Introduction
< Describe the problem in a few sentences >

### Objectives
< What are the objectives / deliverable items from this experiment >

## Materials and Methods



In [ ]:
# EC2 Instance Commands:
# Build/Run `serratus-align`container for indexing
sudo yum install -y docker
sudo yum install -y git
sudo yum install -y less
sudo service docker start

git clone https://github.com/ababaian/serratus.git; cd serratus
sudo docker build -t serratus-base:0 -t serratus-base:latest -f docker/Dockerfile .
sudo docker build -t serratus-align:0 -t serratus-align:latest -f docker/serratus-align/Dockerfile .

sudo docker run --rm --entrypoint /bin/bash -it serratus-align:0


In [ ]:
# local ART install
wget https://www.niehs.nih.gov/research/resources/assets/docs/artsrcmountrainier2016.06.05linux.tgz
tar -xvf artsrcmountrainier2016.06.05linux.tgz
cd art_src_MountRainier_Linux/

sudo yum install gcc-c++ gsl gsl-devel
./configure && make && make install
cp art_illumina /usr/bin/
cd ..

# EMBOSS Tools
wget ftp://emboss.open-bio.org/pub/EMBOSS/EMBOSS-6.6.0.tar.gz
tar -xvf EMBOSS-6.6.0.tar.gz
cd EMBOSS-6.6.0/
./configure --without-x && make && make install
cp emboss/msbar /usr/bin/

# Wuhan SARS-CoV-2 genome
wget ftp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/chromosomes/NC_045512v2.fa.gz
gzip -d NC_045512v2.fa.gz
samtools faidx NC_045512v2.fa

In [ ]:
mkdir sim; cd sim

cp ../NC_045512v2.fa index.fa

# Mutation rate 
# 0, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4
# for a 30kb genome
MU=(0 30 300 1500 3000 4500 6000 7500 9000 10500 12000)

for mu in ${MU[@]}
do
  # Mutate input sequence at mu-rate
  msbar -point 4 -block 0 -codon 0 \
    -count $mu \
    -sequence index.fa \
    -outseq sim.cov."$mu".fa
  
  # Change header with mutation rate 
    sed -i "s/>.*/>mu_$mu/g" sim.cov."$mu".fa
    
  # Simulate reads based on each mutation rate
  art_illumina \
    --seqSys HS20 --paired \
    --in sim.cov."$mu".fa \
    --len 100 --mflen 300 --sdev 1 \
    --fcov 50 \
    --rndSeed 666 \
    --out sim.cov."$mu"_ --noALN \
    > log.tmp
    
  rm log.tmp
done


In [ ]:
# Organize and upload 
mkdir fa; mv *.fa fa/; cd fa; gzip *; cd ..
mkdir fq; mv *.fq fq/; cd fq; gzip *; cd ..

# ../200411/
aws s3 cp --recursive fa s3://serratus-public/notebook/200411/fa/
aws s3 cp --recursive fa s3://serratus-public/notebook/200411/fa/



## Results & Discussion
